In [ ]:
# https://huggingface.co/vincenzodeleo/results/blob/main/README.md # MODELLO FINE-TUNED
# vincenzodeleo/flan-t5-base-sdg-text-classification # MODELLO FINE-TUNED

In [ ]:
from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [ ]:
!huggingface-cli login --token $huggingface_cli_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
experiment_name = "2024.02.29-01_Flan-T5_base_Fine_Tuning_NO-PROMPT---EVALUATION---METRICHE_CON_TRE_CIFRE---NEW"

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"
input_data_path_test = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"

output_data_path = base_path + "/../Flan-T5_FineTuning/data/output/" + experiment_name
log_filename = base_path + "/../Flan-T5_FineTuning/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install -q --upgrade peft

In [ ]:
from peft import PeftModel, PeftConfig
# from transformers import AutoModelForSeq2SeqLM

# # config = PeftConfig.from_pretrained("vincenzodeleo/results")
# config = PeftConfig.from_pretrained("vincenzodeleo/flan-t5-large-sdg-text-classification")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl", device_map="auto")
# # model = PeftModel.from_pretrained(model, "vincenzodeleo/results", device_map="auto")
# model = PeftModel.from_pretrained(model, "vincenzodeleo/flan-t5-large-sdg-text-classification", device_map="auto")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("vincenzodeleo/flan-t5-base-flan-t5-base-sdg-text-classification", device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained("vincenzodeleo/flan-t5-base-flan-t5-base-sdg-text-classification", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
import datasets
import pandas as pd

In [ ]:
from datasets import Dataset, DatasetDict

def convert_to_dataset_dict(df_train, df_test):
    dataset_train = Dataset.from_pandas(df_train)
    dataset_test = Dataset.from_pandas(df_test)
    dataset_dict = DatasetDict({
        'train': dataset_train,
        'test': dataset_test
    })
    return dataset_dict

In [ ]:
train_data = pd.read_excel(input_data_path_train)[['text', 'sdg']].rename(columns={'sdg':'label'})
train_data.head(3)

,text,label
0,This Vitamin Reduces Mental Health Problems By...,0
1,'League Of Legends' unveils new Arena game mod...,0
2,Community remembers Maddi Kingsbury at public ...,0


In [ ]:
test_data = pd.read_excel(input_data_path_test)[['text', 'sdg']].rename(columns={'sdg':'label'})
test_data.head(3)

,text,label
0,L train resumes service after derailing from t...,0
1,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0
2,Mumbai: The Bombay High Court has suspended th...,0


In [ ]:
dataset = convert_to_dataset_dict(train_data, test_data)

In [ ]:
import pandas as pd
from datasets import Dataset
import random

# dataset['train'] = dataset['train'].shuffle(seed=42).select(range(2000))
# dataset['test'] = dataset['test'].shuffle(seed=42).select(range(1000))

dataset['train'] = dataset['train'].shuffle(seed=42)

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
dataset.clear()
train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/5780 [00:00<?, ? examples/s]

Max source length: 340


Map:   0%|          | 0/5780 [00:00<?, ? examples/s]

Max target length: 3


In [ ]:
def preprocess_function(sample, padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/4760 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = dataset['test']['text'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs = inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['label'][i])

  progress_bar.update(1)

  0%|          | 0/1020 [00:00<?, ?it/s]

In [ ]:
str_labels_list = []
for i in range(len(labels_list)): str_labels_list.append(str(labels_list[i]))

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           "       0.00      0.00      0.00         0
           0       0.98      0.98      0.98        60
           1       0.79      0.83      0.81        60
          10       0.81      0.70      0.75        60
          11       0.89      0.95      0.92        60
          12       0.95      0.93      0.94        60
          13       0.88      0.88      0.88        60
          14       0.98      0.98      0.98        60
          15       0.93      0.93      0.93        60
          16       1.00      0.95      0.97        60
           2       0.95      0.93      0.94        60
           3       0.95      0.97      0.96        60
           4       0.90      0.95      0.93        60
           5       0.89      0.95      0.92        60
           6       0.92      0.97      0.94        60
           7       0.95      0.92      0.93        60
           8       0.78      0.72      0.75        60
           9       0.82    

In [ ]:
# str_labels_list2 = []
# predictions_list2 = []
# for i in range(len(labels_list)):
#   # print(str(labels_list[i]), )
#   if predictions_list[i] not in ['color', 'images', 'news', 'on', 'photo']:
#     str_labels_list2.append(str(labels_list[i]))
#     predictions_list2.append(predictions_list[i])

In [ ]:
# from sklearn.metrics import classification_report

# report2 = classification_report(str_labels_list2, predictions_list2, zero_division=0)
# print(report2)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0, digits=4)
print(report)

              precision    recall  f1-score   support

           "     0.0000    0.0000    0.0000         0
           0     0.9833    0.9833    0.9833        60
           1     0.7937    0.8333    0.8130        60
          10     0.8077    0.7000    0.7500        60
          11     0.8906    0.9500    0.9194        60
          12     0.9492    0.9333    0.9412        60
          13     0.8833    0.8833    0.8833        60
          14     0.9833    0.9833    0.9833        60
          15     0.9333    0.9333    0.9333        60
          16     1.0000    0.9500    0.9744        60
           2     0.9492    0.9333    0.9412        60
           3     0.9508    0.9667    0.9587        60
           4     0.9048    0.9500    0.9268        60
           5     0.8906    0.9500    0.9194        60
           6     0.9206    0.9667    0.9431        60
           7     0.9483    0.9167    0.9322        60
           8     0.7818    0.7167    0.7478        60
           9     0.8197    

In [ ]:
sdg_list = [str(x) for x in range(17)]
print(sdg_list)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16']


In [ ]:
predictions_list_corrected = []
for l, p in zip(str_labels_list, predictions_list):
  # print(l,p, type(l), type(p))
  if p not in sdg_list:
    #print(l,p)
    p_new = '0' #random.choice([x for x in sdg_list if x != str(p)])
    print(l,p, p_new)
    # break
    predictions_list_corrected.append(p_new)
  else:
    predictions_list_corrected.append(p)

0 " 0


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list_corrected, zero_division=0, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9836    1.0000    0.9917        60
           1     0.7937    0.8333    0.8130        60
          10     0.8077    0.7000    0.7500        60
          11     0.8906    0.9500    0.9194        60
          12     0.9492    0.9333    0.9412        60
          13     0.8833    0.8833    0.8833        60
          14     0.9833    0.9833    0.9833        60
          15     0.9333    0.9333    0.9333        60
          16     1.0000    0.9500    0.9744        60
           2     0.9492    0.9333    0.9412        60
           3     0.9508    0.9667    0.9587        60
           4     0.9048    0.9500    0.9268        60
           5     0.8906    0.9500    0.9194        60
           6     0.9206    0.9667    0.9431        60
           7     0.9483    0.9167    0.9322        60
           8     0.7818    0.7167    0.7478        60
           9     0.8197    0.8333    0.8264        60

    accuracy              